# Text 생성 및 Prompt 예제

- https://platform.openai.com/docs/examples 참조

### Text 생성 API 비교
| 특징                                                                                            | **Responses API**                                                         | **Chat Completions API**                                              |
| --------------------------------------------------------------------------------------------- | ------------------------------------------------------------------------- | --------------------------------------------------------------------- |
| **상태 유지(Stateful)**                                                                           | ✅ 서버 측에서 **대화 상태를 관리(저장 및 이어쓰기 가능)**                                      |                                                                                                                |                                                                       |
| **내장 도구 지원(Built-in Tools)**                                                                  | ✅ `web_search`, `file_search`, `computer_use_preview` 등 자동 호출 가능          | ❌ 기본 제공 도구 없음. **Function calling은 가능하지만, 호출/실행은 직접 구현 필요**           |
| **구조 & 사용 방식**                                                                                | `instructions`, `input`, `tools`, `store`, `previous_response_id` 기반 API  | `messages=[{role, content}, ...]`, `temperature`, `max_tokens`, 등의 구조 |
| **응답 형식 (Output)**                                                                            | `response.output_text`, `response.output` 구조로 단일 메세지 응답 및 도구 호출 결과 포함     | `choices[0].message.content` 형태로 단일 응답만 반환                            |
| **사용 목적**                                                                                     | ✅ **에이전트/도구 사용, 파일/웹 검색, 상태 관리**가 필요한 복잡한 워크플로우에 적합                       | ✅ **챗봇, 단순 대화, instruction-following**에 적합                            |
| **미래 확장성**                                                                                    | 🔧 OpenAI Roadmap 중심 – Responses API가 Chat API의 기능을 통합 발전시키는 방향           | 📦 **기존 표준 API**, 계속 지원되며 업계 표준 유지                                    |

[1]: https://simonwillison.net/2025/Mar/11/responses-vs-chat-completions/?utm_source=chatgpt.com "OpenAI API: Responses vs. Chat Completions"
[2]: https://cookbook.openai.com/examples/responses_api/responses_example?utm_source=chatgpt.com "Web Search and States with Responses API - OpenAI Cookbook"


In [ ]:
# !pip install -q openai python-dotenv

## Chat Completions API - Legacy

In [ ]:
# 응답 결과 출력

## Responses API - New

## 문법 수정

SYSTEM : 당신은 문장을 받게 될 것이며, 당신의 임무는 그것을 표준 한국어로 변환하는 것입니다.  
USER :   
안갔어 시장에 그녀는.

In [ ]:
# Responses API 호출 예시
# 응답 결과 출력

## 프롬프트 엔지니어링(Prompt Engineering)
모델이 요구사항에 맞는 결과물을 안정적으로 생성하도록 효과적인 지시문(prompt)을 작성하는 과정입니다. 
- 일부 프롬프트 엔지니어링 기법은 모든 모델에서 공통적으로 유효
- 모델마다 프롬프트를 해석하는 방식이 다르기 때문에, 최적의 결과를 얻으려면 모델별로 프롬프트를 조정해야 합니다.
- 프로덕션 환경에서는 특정 모델의 특정 버전을 고정하여 일관된 동작을 유지하세요.

### 모델 및 API 선택 
Reasoning 모델(o3, GPT-5 등) 은 일반 Chat 모델과 다르게 동작하며, 특히 중요한 점은, Reasoning 모델은 Responses API를 사용할 때 훨씬 더 높은 성능과 “지적 추론 능력”을 보여준다는 것입니다.  
$\rightarrow$ 텍스트 생성(text generation) 관련 애플리케이션을 개발할 때는 기존의 Chat Completions API 대신 Responses API를 사용하는 것을 강력히 권장 

### 메시지 역할(roles)과 지시문(instruction) 

instructions 파라미터와 메시지 역할(message roles) 을 함께 사용하면, 모델에게 서로 다른 수준의 권한(authority) 을 가진 지시문을 전달할 수 있습니다.  
instructions 파라미터는 모델이 응답을 생성할 때 따라야 할 말투(tone), 목표(goals), 올바른 응답의 예시와 같은 전반적인 행동 지침을 제공합니다.  
이렇게 전달된 지시문은 입력 프롬프트(input 매개변수)에 포함된 지시보다 우선순위가 더 높습니다.

---------------
위의 예시는 `input` 배열에서 다음과 같은 메시지를 사용하는 것과 거의 동일합니다.

## 구조화되지 않은 데이터의 구문 분석
SYSTEM : 구조화되지 않은 데이터가 제공되며 이를 CSV 형식으로 구문 분석하는 작업이 수행됩니다.  
USER :   
최근 발견된 행성 구크럭스(Goocrux)에서는 많은 과일이 발견됐다. 그곳에서 자라는 네오스키즐이 있는데, 보라색이고 사탕 맛이 납니다. 회색 빛이 도는 파란색 과일이고 매우 시큼하며 레몬과 약간 비슷한 로헤클(loheckles)도 있습니다. 포유닛은 밝은 녹색을 띠며 단맛보다 풍미가 더 좋습니다. 네온 핑크색 맛과 솜사탕 같은 맛이 나는 루프노바도 많이 있습니다. 마지막으로 글로울(glowls)이라는 과일이 있는데, 이 과일은 신맛과 부식성이 있는 매우 신맛과 쓴맛이 나며 옅은 오렌지색을 띠고 있습니다.

## 대화 상태를 수동으로 관리

| 역할       | 설명                                                                                      |
|------------|-------------------------------------------------------------------------------------------|
| developer  | 애플리케이션 개발자가 제공한 지침. **user 메시지보다 우선적으로 처리**됩니다.              |
| user       | 최종 사용자가 입력한 지침. developer 메시지보다 **우선순위가 낮습니다.**                  |
| assistant  | 모델이 생성한 응답 메시지. 응답 결과는 이 역할로 반환됩니다.                              |


## Keyword 추출 

reasoning 객체의 속성 effort (예: "low")는 reasoning 토큰을 얼마나 할당해서 모델이 내부 사고를 할지 정도를 조절

## Python 버그 수정 
SYSTEM : Python 코드 조각이 제공되며, 귀하의 임무는 그 안의 버그를 찾아 수정하는 것입니다.  
USER : 
```Python
    import Random
    a = random.randint(1,12)
    b = random.randint(1,12)
    for i in range(10):
        question = "What is "+a+" x "+b+"? "
        answer = input(question)
        if answer = a*b
            print (Well done!)
        else:
            print("No.")
```

## Program Code 설명 

In [ ]:
class Log:
    def __init__(self, path):
        # Check that the file is newline-terminated
    def log(self, event):
    def state(self):

## 감성 분류기
- 한개의 text 감성 분석

- 여러개의 text 감성 분석

In [ ]:
    # 감정 분석 결과 저장
# 결과 출력

## 회의록 요약

SYSTEM : 회의록이 제공되며 귀하의 임무는 다음과 같이 회의를 요약하는 것입니다.  

 -토론의 전반적인 요약  
 -행동항목(무엇을 해야 하는지, 누가 하는지)  
 -해당하는 경우 다음 회의에서 더 자세히 논의해야 할 주제 목록입니다.  

## 이모티콘 번역
SYSTEM : 텍스트가 제공되며, 귀하의 임무는 이를 이모티콘으로 번역하는 것입니다. 일반 텍스트를 사용하지 마십시오. 이모티콘만으로 최선을 다하세요.  
USER : 인공지능은 큰 가능성을 지닌 기술이다.

## 번역

----------------
## GPT-5 parameters

- verbosity 제어 — 모델이 얼마나 자세하게 응답할지를 조정 (high, medium, low) 
- reasoning effort — 최소한의 추론 단계를 거쳐 빠른 응답 제공 (high, medium, low)  
- custom tools — 개발자가 직접 정의한 맞춤형 도구 사용 가능  
- allowed tools list — 모델이 사용할 수 있는 도구 목록을 제한하여 안전성 강화

### Token 수 계산

In [ ]:
# pip install tiktoken

In [ ]:
def serialize_messages_chatml(messages: List[Dict[str, Any]]) -> str:
            # 일부 모델은 name 지원. 표기는 role 다음 줄에 name: ... 형태로 덧붙여 직렬화 가능
    # 최종 assistant 답변의 프롬프트 마커(모델에 따라 선택적으로 사용)